## 统计分析

通过指定统计分析字段，得到每个特征的p_value，所有的p_value计算都是基于Ttest计算。支持指定不同的分组`group`，例如train、val、test等分组统计。

对于两大类不同的特征

1. 离散特征，统计数量以及占比。
2. 连续特征，统计均值、方差。

In [6]:
import pandas as pd
import os
from onekey_algo import OnekeyDS as okds

mydir = r'D:\20220421-FANBIN'
labelf = os.path.join(mydir, 'label.csv')
label_data = pd.read_csv(labelf)
label_data[['ID']] = label_data[['ID']].applymap(lambda x: f"{x}.nii.gz")
label_data[['ID']] = label_data[['ID']].applymap(lambda x: x.replace('-m', '').replace('-f', ''))
class_mapping = {'I': 0, "IIA": 1, "IIB": 2, 'III':3, 'IV':4, 'V':5, 'VI':5}
label_data[['label']] = label_data[['label']].applymap(lambda x: class_mapping[x])

data = pd.read_csv(r'rad_features1.csv')
data[['ID']] = data[['ID']].applymap(lambda x: x.replace('-m', '').replace('-f', ''))
data.columns = [c if c == 'ID' else c.replace('-', '_') for c in data.columns]

combined_data = pd.merge(label_data, data, on='ID', how='inner')
print(combined_data.columns)
combined_data

Index(['ID', 'label', 'exponential_firstorder_10Percentile',
       'exponential_firstorder_90Percentile', 'exponential_firstorder_Energy',
       'exponential_firstorder_Entropy',
       'exponential_firstorder_InterquartileRange',
       'exponential_firstorder_Kurtosis', 'exponential_firstorder_Maximum',
       'exponential_firstorder_Mean',
       ...
       'wavelet_LLL_glszm_LargeAreaLowGrayLevelEmphasis',
       'wavelet_LLL_glszm_LowGrayLevelZoneEmphasis',
       'wavelet_LLL_glszm_SizeZoneNonUniformity',
       'wavelet_LLL_glszm_SizeZoneNonUniformityNormalized',
       'wavelet_LLL_glszm_SmallAreaEmphasis',
       'wavelet_LLL_glszm_SmallAreaHighGrayLevelEmphasis',
       'wavelet_LLL_glszm_SmallAreaLowGrayLevelEmphasis',
       'wavelet_LLL_glszm_ZoneEntropy', 'wavelet_LLL_glszm_ZonePercentage',
       'wavelet_LLL_glszm_ZoneVariance'],
      dtype='object', length=1908)


,ID,label,exponential_firstorder_10Percentile,exponential_firstorder_90Percentile,exponential_firstorder_Energy,exponential_firstorder_Entropy,exponential_firstorder_InterquartileRange,exponential_firstorder_Kurtosis,exponential_firstorder_Maximum,exponential_firstorder_Mean,...,wavelet_LLL_glszm_LargeAreaLowGrayLevelEmphasis,wavelet_LLL_glszm_LowGrayLevelZoneEmphasis,wavelet_LLL_glszm_SizeZoneNonUniformity,wavelet_LLL_glszm_SizeZoneNonUniformityNormalized,wavelet_LLL_glszm_SmallAreaEmphasis,wavelet_LLL_glszm_SmallAreaHighGrayLevelEmphasis,wavelet_LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet_LLL_glszm_ZoneEntropy,wavelet_LLL_glszm_ZonePercentage,wavelet_LLL_glszm_ZoneVariance
0,17026055.nii.gz,1,1.201357,2.903185,9.395387e+08,3.875978e-01,0.671840,122.104286,64.023255,2.415562,...,0.001112,0.001111,848.796685,0.937897,0.975813,406173.333702,0.001111,8.944627,0.967914,0.034260
1,17098412.nii.gz,5,0.883613,2.467127,6.401484e+08,1.820151e-01,0.566603,204.074528,50.460506,1.677484,...,0.001602,0.001601,608.318471,0.968660,0.988057,385954.363854,0.001601,8.715175,0.984326,0.015670
2,17113727.nii.gz,3,0.936020,4.890556,5.671943e+08,6.415838e-01,1.412400,231.369202,231.666179,3.616451,...,0.001829,0.001829,520.667883,0.950124,0.980586,715696.804187,0.001829,8.749868,0.973357,0.030273
3,17141660.nii.gz,1,0.537851,1.116522,8.536338e+08,9.978644e-02,0.246245,306.450106,41.104725,0.956734,...,0.001225,0.001224,770.650061,0.936391,0.975057,269826.311293,0.001224,8.681159,0.965962,0.038856
4,18001036.nii.gz,0,0.739813,2.077098,5.033043e+08,1.179667e-01,0.442961,30.919300,10.278669,1.297672,...,0.002087,0.002084,455.053498,0.936324,0.975309,129192.797840,0.002084,8.209459,0.968127,0.031838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,zhaolei.nii.gz,1,1.221794,2.595526,1.756409e+08,-3.203427e-16,0.654361,3.990087,3.744069,1.829344,...,0.005905,0.005905,160.294118,0.942907,0.977941,293617.373529,0.005904,7.224039,0.971429,0.028547
120,zhaorui.nii.gz,0,1.455344,2.942381,3.715565e+07,-3.203427e-16,0.830143,4.485463,4.282266,2.100978,...,0.027045,0.027045,37.000000,1.000000,1.000000,135939.108108,0.027045,5.155399,1.000000,0.000000
121,zhaoshuping.nii.gz,2,0.916472,1.588933,1.404375e+08,1.080232e-01,0.361182,64.984785,25.023852,1.557034,...,0.007392,0.007389,130.102941,0.956639,0.982435,226053.216503,0.007389,6.890736,0.971429,0.043253
122,zhouhcuanhai.nii.gz,5,0.890511,2.295600,3.661399e+08,1.856908e-01,0.606133,44.886624,13.306170,1.559592,...,0.003019,0.002930,337.411268,0.950454,0.980595,101333.978873,0.002908,7.904567,0.972603,0.033009


### 输出格式
支持两种格式数据，分别对应`pretty`参数的`True`和`False`, 当为`True`时，输出的是表格模式，反之则为dict数据。

```python
def clinic_stats(data: DataFrame, stats_columns: Union[str, List[str]], label_column='label',
                 group_column: str = None, continuous_columns: Union[str, List[str]] = None,
                 pretty: bool = True) -> Union[dict, DataFrame]:
    """

    Args:
        data: 数据
        stats_columns: 需要统计的列名
        label_column: 二分类的标签列，默认`label`
        group_column: 分组统计依据，例如区分训练组、测试组、验证组。
        continuous_columns: 那些列是连续变量，连续变量统计均值方差。
        pretty: bool, 是否对结果进行格式美化。

    Returns:
        stats DataFrame or json

    """
```

In [7]:
from onekey_algo.custom.components.stats import clinic_stats
stats = clinic_stats(combined_data, 
                     stats_columns=list(data.columns[1:]),
                     label_column='label', 
                     group_column=None, 
                     continuous_columns=list(data.columns[1:]), 
                     pretty=True)
stats.to_csv('features_stats.csv', header=True, index=False, encoding='gbk')
stats

Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anov

Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anov

Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anov

Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anova...
Using anov

,feature_name,-label=ALL,-label=0,-label=1,-label=2,-label=3,-label=4,-label=5,pvalue
0,exponential_firstorder_10Percentile,0.9441±0.2285,0.9170±0.2356,0.9959±0.2822,0.8861±0.1187,0.8914±0.1149,1.0117±0.1704,0.9682±0.2416,0.379642
1,exponential_firstorder_90Percentile,2.1869±0.8545,2.0387±0.6944,2.4489±1.1124,1.7849±0.4117,2.0293±1.1834,2.5181±0.3977,2.3361±0.9600,0.160159
2,exponential_firstorder_Energy,357730731.8367±240963195.3086,355026730.4626±246581199.7432,474750452.1643±264962270.5783,146735961.1605±80412010.4893,221159450.2975±159193490.8011,479932753.3417±197693347.2962,350955917.2621±224470233.8808,0.795842
3,exponential_firstorder_Entropy,0.1325±0.1426,0.1298±0.1260,0.1700±0.1720,0.0295±0.0597,0.1101±0.2230,0.1568±0.0784,0.1432±0.1499,0.821454
4,exponential_firstorder_InterquartileRange,0.5366±0.2883,0.4663±0.2064,0.6362±0.3664,0.4173±0.1491,0.4952±0.3862,0.6107±0.1284,0.6180±0.3560,0.044773
...,...,...,...,...,...,...,...,...,...
1901,wavelet_LLL_glszm_SmallAreaHighGrayLevelEmphasis,245830.8796±128856.9312,227131.7715±107460.4432,286305.2623±125869.2138,152593.4503±47264.5754,224138.6917±206660.1784,377348.8217±166778.3318,245656.9148±114276.4429,0.267800
1902,wavelet_LLL_glszm_SmallAreaLowGrayLevelEmphasis,0.0067±0.0098,0.0085±0.0132,0.0030±0.0022,0.0090±0.0039,0.0072±0.0051,0.0026±0.0013,0.0062±0.0091,0.323362
1903,wavelet_LLL_glszm_ZoneEntropy,7.5851±1.0602,7.4704±1.2443,8.0553±0.6748,6.7924±0.6197,7.1547±0.8958,8.2731±0.4342,7.6262±0.9943,0.537069
1904,wavelet_LLL_glszm_ZonePercentage,0.9759±0.0136,0.9758±0.0154,0.9717±0.0077,0.9755±0.0145,0.9725±0.0143,0.9802±0.0064,0.9786±0.0142,0.234886


In [5]:
clinic_stats(combined_data, 
                     stats_columns=['Age', 'Gender'],
                     label_column='label', 
                     group_column='group', 
                     continuous_columns=['Age'], 
                     pretty=False)

{'Age': {'test': {'mean': 48.96774193548387,
   'std': 11.847598549826436,
   '__pvalue__': 0.22406284497183407,
   'mean | label=5': 49.666666666666664,
   'std | label=5': 11.413442367080435,
   'mean | label=1': 53.0,
   'std | label=1': 10.64581294844754,
   'mean | label=0': 50.86666666666667,
   'std | label=0': 10.50759362601751,
   'mean | label=4': 29.666666666666668,
   'std | label=4': 14.224392195567912,
   'mean | label=3': 52.0,
   'std | label=3': 1.0},
  'train': {'mean': 49.157303370786515,
   'std': 11.367716325794662,
   '__pvalue__': 0.1882081098007302,
   'mean | label=1': 49.142857142857146,
   'std | label=1': 9.062663412698727,
   'mean | label=3': 53.0,
   'std | label=3': 2.943920288775949,
   'mean | label=0': 46.911764705882355,
   'std | label=0': 12.235981829706123,
   'mean | label=4': 46.333333333333336,
   'std | label=4': 10.614455552060438,
   'mean | label=2': 52.25,
   'std | label=2': 14.02803315813426,
   'mean | label=5': 51.47826086956522,
   's